# Do the realistic lossy polymer wl study

We have 6 and 8 tube fibers. Now doing the 6 tube

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ngsolve.webgui import Draw
from ngsolve import CF

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from scipy.optimize import newton

# Mode Profiles

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
n_poly = 1.5 - 2.4e-3

ts = [15e-6, .42e-6, 12.48e-6, 10e-6, 60e-6, 10e-6]
ns = [lambda x:n_air, lambda x:n_glass,
      lambda x:n_air, lambda x:n_glass,
      lambda x:n_poly, lambda x:n_air]

mats = ['air','glass','air','glass','polymer','air']
maxhs = [.2, .1, .2, .1, .1, 10]
scale = 15e-6
wl = 1.8e-6

In [ ]:
A = BraggExact(ts=ts, ns=ns, mats=mats, maxhs=maxhs, scale=scale, wl=wl)


In [ ]:
Draw(CF(list(A.ns)), A.mesh)

In [ ]:
# k_low = A.k0 * A.ns[0] * A.scale
k_low, k_high = A.k0 * np.min(A.ns.real) * A.scale, A.k0 * np.max(A.ns.real) * A.scale

outer = 'h2'
Ktype = 'kappa'
nu = 1

In [ ]:
%matplotlib inline
plotlogf(A.determinant, .997*k_low, 1.00001*k_low, -.03,.03, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.9999*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-16)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


In [ ]:
Fs = A.all_fields(beta1, nu=nu, outer=outer, Ktype=Ktype)

In [ ]:
Draw(10*Fs['Ez'], A.mesh)

# Wavelength Studies

## N2 reference fiber

In [ ]:
n = 300
wls = np.linspace(1e-6, 2e-6, n+1)
betasN2 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

n_air = 1.00027717
n_glass = 1.4388164768221814

In [ ]:
ts = [15e-6, .42e-6, 12.48e-6, 10e-6]
ns = [lambda x:n_air, lambda x:n_glass,
      lambda x:n_air, lambda x:n_glass]

mats = ['air','glass','air','glass']
maxhs = [.2, .1, .2, .1]
scale = 15e-6


In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
    
    A = BraggExact(ts=ts, ns=ns , mats=mats,maxhs=maxhs, wl=wl, scale=scale,
                  no_mesh=True)
    k_low = A.ks[0] * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.99999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .99999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betasN2[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betasN2.imag, color='orange', linewidth=.9)
plt.yscale('log')


# N3 Model

In [ ]:
n = 300
betasN3 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

n_air = 1.00027717
n_glass = 1.4388164768221814


In [ ]:
ts = [15e-6, .42e-6, 12.48e-6, 70e-6, 10e-6]
ns = [lambda x:n_air, lambda x:n_glass,
      lambda x:n_air, lambda x:n_glass,
      lambda x:n_air]

mats = ['air','glass','air','glass','air']
maxhs = [.2, .01, .2, .01, 10]


Having a hard time getting the fundamental, it's jumping to a higher order mode, let's look where it is.

In [ ]:
A = BraggExact(ts=ts, ns=ns , mats=mats, maxhs=maxhs, wl=wls[0], scale=scale,
              no_mesh=True)

In [ ]:
%matplotlib inline
k_low = A.ks[0] * A.scale

plotlogf(A.determinant, .9995*k_low, 1.00001*k_low, -.03,.03, nu, outer,
         iref=100, rref=100, levels=100)

Lower than I thought, okay

In [ ]:
guess = np.array(.9998*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-16)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
    
    A = BraggExact(ts=ts, ns=ns , mats=mats, maxhs=maxhs, wl=wl, scale=scale,
                  no_mesh=True)
    k_low = A.ks[0] * A.scale
    guess = np.array(.9999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999995
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999995
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betasN3[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betasN2.imag, color='orange', linewidth=.9)
plt.plot(wls, -betasN3.imag, color='green', linewidth=.9)
plt.yscale('log')


# With avg loss from refindx site of PVC

In [ ]:
n = 300
betas1 = np.zeros_like(wls, dtype=complex)

n_air = 1.00027717
n_glass = 1.4388164768221814
n_poly = 1.5 - 1.3675171717171715e-05j

In [ ]:
ts = [15e-6, .42e-6, 12.48e-6, 10e-6, 60e-6, 10e-6]
ns = [lambda x:n_air, lambda x:n_glass,
      lambda x:n_air, lambda x:n_glass,
      lambda x:n_poly, lambda x:n_air]

mats = ['air','glass','air','glass','polymer','air']
maxhs = [.2, .01, .2, .01, .01, 10]
scale = 15e-6


Having a hard time getting the fundamental, it's jumping to a higher order mode, let's look where it is.

In [ ]:
A = BraggExact(ts=ts, ns=ns , mats=mats, maxhs=maxhs, wl=wls[0], scale=scale,
              no_mesh=True)

In [ ]:
%matplotlib inline
k_low = A.ks[0].real * A.scale

plotlogf(A.determinant, .9995*k_low, 1.00001*k_low, -.008,.008, nu, outer,
         iref=100, rref=100, levels=100)

Lower than I thought, okay

In [ ]:
guess = np.array(.9999*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-16)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
    
    A = BraggExact(ts=ts, ns=ns , mats=mats,maxhs=maxhs, wl=wl, scale=scale,
                  no_mesh=True)
    k_low = A.ks[0] * A.scale
    guess = np.array(.9999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999995
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999995
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betasN2.imag, color='orange', linewidth=.9)
plt.plot(wls, -betasN3.imag, color='green', linewidth=.9)
plt.plot(wls, -betas1.imag, color='blue', linewidth=.9)
plt.yscale('log')


# With varying extinction coeff

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
info = np.genfromtxt('PMMA.csv', delimiter=',', skip_header=1)

In [ ]:
L, R = .4, 2

r, c = (R - L) / 2, (R + L) / 2  # center c and radius r to find range of coeffs we want to view

In [ ]:
pmma = info[np.where(np.abs(info[:,0]-c)<r)]

In [ ]:
plt.plot(pmma[:,0], pmma[:,2])
plt.yscale('log')

In [ ]:
info = np.genfromtxt('PVC.csv', delimiter=',', skip_header=1)

In [ ]:
L, R = 1, 2

r, c = (R - L) / 2, (R + L) / 2  # center c and radius r to find range of coeffs we want to view

In [ ]:
pvc= info[np.where(np.abs(info[:,0]-c)<r)]

In [ ]:
np.mean(pvc[:,2])

In [ ]:
plt.plot(pvc[:,0], pvc[:,2])
plt.yscale('log')